In [1]:
%reload_ext xmemo_kernel_extension

In [2]:
import json
import pandas as pd
import pickle
import openai
import functools
import numpy as np
import concurrent.futures

In [3]:
d = pd.read_json('data/squad2.json')['data']
all_paragraphs = [paragraph['context'] for ent in d for paragraph in ent['paragraphs']]

In [4]:
%%xmemo input=all_paragraphs output=all_qa

# Ok. list comprehension is still possible but it gets a little obnoxious.
all_qa = []
paragraph_id = 0
for ent in d:
    for paragraph in ent['paragraphs']:
        for qa in paragraph['qas']:
            if len(qa['answers']) > 0 and qa['is_impossible'] == False:
                all_qa.append((paragraph_id, qa['question'], qa['answers'][0]['text']))
        paragraph_id += 1

Loading from e06196685b0c2069ecd0b9336662b29b395d8b8fc7c1b9eb2117f1c1f2804e7a.pickle


In [5]:
all_qa

[(0, 'When did Beyonce start becoming popular?', 'in the late 1990s'),
 (0,
  'What areas did Beyonce compete in when she was growing up?',
  'singing and dancing'),
 (0,
  "When did Beyonce leave Destiny's Child and become a solo singer?",
  '2003'),
 (0, 'In what city and state did Beyonce  grow up? ', 'Houston, Texas'),
 (0, 'In which decade did Beyonce become famous?', 'late 1990s'),
 (0, 'In what R&B group was she the lead singer?', "Destiny's Child"),
 (0, 'What album made her a worldwide known artist?', 'Dangerously in Love'),
 (0, "Who managed the Destiny's Child group?", 'Mathew Knowles'),
 (0, 'When did Beyoncé rise to fame?', 'late 1990s'),
 (0, "What role did Beyoncé have in Destiny's Child?", 'lead singer'),
 (0,
  'What was the first album Beyoncé released as a solo artist?',
  'Dangerously in Love'),
 (0, 'When did Beyoncé release Dangerously in Love?', '2003'),
 (0,
  'How many Grammy awards did Beyoncé win for her first solo album?',
  'five'),
 (0, "What was Beyoncé's

In [6]:
from sklearn.neighbors import NearestNeighbors
par_embeddings_arr = np.load('data/openai_paragraphs_embeddings.npy')
qn_embeddings_arr = np.load('data/openai_qn_embeddings.npy')

In [10]:
%%xmemo input=par_embeddings_arr,qn_embeddings_arr output=knn
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree', n_jobs=-1)
nbrs.fit(par_embeddings_arr)
knn = nbrs.kneighbors(qn_embeddings_arr)

Loading from 5bf1df97489e86f817f0fc32db826845e3c3db6976a0e95bccdaf62f23b3bfce.pickle


In [11]:
knn

(array([[0.53342373, 0.54183945, 0.5480534 , ..., 0.55267631, 0.55477935,
         0.55652812],
        [0.51625546, 0.56791918, 0.57577417, ..., 0.60289394, 0.60317292,
         0.60357273],
        [0.47841394, 0.49049756, 0.51447069, ..., 0.54367424, 0.5441335 ,
         0.54597068],
        ...,
        [0.57681319, 0.59904399, 0.59999977, ..., 0.6253042 , 0.62548659,
         0.62892828],
        [0.49743956, 0.57099157, 0.5722643 , ..., 0.59779   , 0.60029111,
         0.60105112],
        [0.6205644 , 0.67376102, 0.68988245, ..., 0.70489665, 0.70647834,
         0.70827299]]),
 array([[   23,    45,    26, ...,    50,     0,    14],
        [    4,     5,     3, ...,    64,    42,    20],
        [    1,    11,     8, ...,     7,    23,    36],
        ...,
        [18979, 18929, 18925, ..., 18966, 18972, 18934],
        [18979, 18932, 18940, ..., 18978, 18972, 18941],
        [18979, 18255,  3350, ...,   126, 13786, 11440]]))